In [ ]:
# ## consume to mongodb
# from kafka import KafkaConsumer
# import sys, json, pymongo
# # from elasticsearch import Elasticsearch
# # import time, datetime



# if __name__ == "__main__":

#     # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.xyz

#     # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
#     consumer = KafkaConsumer(
#         # 指定Kafka集群伺服器
#         bootstrap_servers=["kafka:9092"],
#         # ConsumerGroup的名稱, 可以不指定
#         #group_id="cg_001",
#         # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
#         # key_deserializer=bytes.decode,
#         # 指定msgValue的反序列化器
#         #value_deserializer=bytes.decode,
#         value_deserializer=lambda m: json.loads(m.decode('ascii')),
#         # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
#         auto_offset_reset="earliest",
#     )
   
#     # 讓Consumer向Kafka集群訂閱指定的topic
#     consumer.subscribe(topics="pace")
    
#     #es = Elasticsearch('http://elasticsearch:9200')
    
    
#     # 持續的拉取Kafka有進來的訊息
#     try:
#         print("Now listening for incoming messages ...")
#         # 持續監控是否有新的record進來
#         for record in consumer:
#             topic = record.topic
#             partition = record.partition
#             offset = record.offset
#             timestamp = record.timestamp
#             # 取出msgKey與msgValue
#             msgKey = record.key
#             msgValue = record.value
#             # 秀出metadata與msgKey & msgValue訊息
#             print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
#                                                                               record.offset, record.key, record.value))
#             j = {"device_id": msgValue["device_id"], "timestamp": msgValue["timestamp"], "speed": msgValue["speed"]}
#             print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             #result = collection.insert_many()
#             print(result)
#             #res = es.index(index="temp_humidity1", doc_type='tweet', id=datetime.now(), body=j)
#             #print(res['result'])

#     except:
#         # 錯誤處理
#         e_type, e_value, e_traceback = sys.exc_info()
#         print("type ==> %s" % (e_type))
#         print("value ==> %s" % (e_value))
#         print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
#         print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
#         print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
#     finally:
#         consumer.close()

In [ ]:
## consume to Elasticsearch
from kafka import KafkaConsumer
import sys, json
# import pymongo
from elasticsearch import Elasticsearch
import time, datetime




if __name__ == "__main__":

    # 與 MongoDB連線
#     client = pymongo.MongoClient(host="mongodb", port=27017)
#     # 指定為 test 資料庫
#     db = client.test
#     # 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
#     collection = db.xyz

    # 設定要連線到Kafka集群的相關設定, 產生一個Kafka的Consumer的實例
    consumer = KafkaConsumer(
        # 指定Kafka集群伺服器
        bootstrap_servers=["kafka:9092"],
        # ConsumerGroup的名稱, 可以不指定
        #group_id="cg_001",
        # 指定msgKey的反序列化器, 若Key為None, 無法反序列化
        # key_deserializer=bytes.decode,
        # 指定msgValue的反序列化器
        #value_deserializer=bytes.decode,
        value_deserializer=lambda m: json.loads(m.decode('ascii')),
        # 是否從這個ConsumerGroup尚未讀取的partition / offset開始讀
        auto_offset_reset="earliest",
    )
   
    # 讓Consumer向Kafka集群訂閱指定的topic
    consumer.subscribe(topics="pace3")
    
    es = Elasticsearch('http://elasticsearch:9200')
    
    
    # 持續的拉取Kafka有進來的訊息
    try:
        print("Now listening for incoming messages ...")
        # 持續監控是否有新的record進來
        for record in consumer:
            topic = record.topic
            partition = record.partition
            offset = record.offset
            timestamp = record.timestamp
            # 取出msgKey與msgValue
            msgKey = record.key
            msgValue = record.value
            # 秀出metadata與msgKey & msgValue訊息
            print("topic=%s, partition=%s, offset=%s : (key=%s, value=%s)" % (record.topic, record.partition, 
                                                                              record.offset, record.key, record.value))
            #輸入的timestamp須為時間格式
            j = {"device_id": msgValue["device_id"], "timestamp": msgValue["timestamp"],"start": msgValue["start"],
                "time_total": msgValue["time_total"], "diff": msgValue["diff"], "pace": msgValue["pace"], "distance": msgValue["distance"]}
            print("json=",  j)
            
#             # 將資料存入 mongodb
#             # 存入單筆
#             result = collection.insert_one(j)
#             # 存入多筆
#             # result = collection.insert_many()
#             print(result)

            res = es.index(index="pace3", doc_type='tweet', id=datetime.datetime.now(), body=j)
            print(res['result'])

    except:
        # 錯誤處理
        e_type, e_value, e_traceback = sys.exc_info()
        print("type ==> %s" % (e_type))
        print("value ==> %s" % (e_value))
        print("traceback ==> file name: %s" % (e_traceback.tb_frame.f_code.co_filename))
        print("traceback ==> line no: %s" % (e_traceback.tb_lineno))
        print("traceback ==> function name: %s" % (e_traceback.tb_frame.f_code.co_name))
    finally:
        consumer.close()

Now listening for incoming messages ...
topic=pace3, partition=0, offset=0 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:17:23', 'start': '2019-10-16 19:17:22', 'pace': 212.01, 'time_total': '0H 0M 0S', 'diff': 0, 'distance': 0.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:17:23', 'start': '2019-10-16 19:17:22', 'time_total': '0H 0M 0S', 'diff': 0, 'pace': 212.01, 'distance': 0.0}
created
topic=pace3, partition=0, offset=1 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:17:26', 'start': '2019-10-16 19:17:22', 'pace': 133.63, 'time_total': '0H 0M 4S', 'diff': 4, 'distance': 0.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:17:26', 'start': '2019-10-16 19:17:22', 'time_total': '0H 0M 4S', 'diff': 4, 'pace': 133.63, 'distance': 0.0}
created
topic=pace3, partition=0, offset=2 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:17:30', 'start': '2019-10-16 19:17:22', 'pace': 73.17, 'time_total': '0H 0M 8S

created
topic=pace3, partition=0, offset=33 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:19:26', 'start': '2019-10-16 19:17:22', 'pace': 49.18, 'time_total': '0H 2M 4S', 'diff': 124, 'distance': 0.3})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:19:26', 'start': '2019-10-16 19:17:22', 'time_total': '0H 2M 4S', 'diff': 124, 'pace': 49.18, 'distance': 0.3}
created
topic=pace3, partition=0, offset=34 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:19:29', 'start': '2019-10-16 19:17:22', 'pace': 126.85, 'time_total': '0H 2M 7S', 'diff': 127, 'distance': 0.32})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:19:29', 'start': '2019-10-16 19:17:22', 'time_total': '0H 2M 7S', 'diff': 127, 'pace': 126.85, 'distance': 0.32}
created
topic=pace3, partition=0, offset=35 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:19:33', 'start': '2019-10-16 19:17:22', 'pace': 83.45, 'time_total': '0H 2M 11S', 'diff': 131, 'dis

created
topic=pace3, partition=0, offset=89 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:22:58', 'start': '2019-10-16 19:17:22', 'pace': 322.58, 'time_total': '0H 5M 36S', 'diff': 336, 'distance': 1.560000000000001})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:22:58', 'start': '2019-10-16 19:17:22', 'time_total': '0H 5M 36S', 'diff': 336, 'pace': 322.58, 'distance': 1.560000000000001}
created
topic=pace3, partition=0, offset=90 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:23:01', 'start': '2019-10-16 19:17:22', 'pace': 215.83, 'time_total': '0H 5M 39S', 'diff': 339, 'distance': 1.590000000000001})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:23:01', 'start': '2019-10-16 19:17:22', 'time_total': '0H 5M 39S', 'diff': 339, 'pace': 215.83, 'distance': 1.590000000000001}
created
topic=pace3, partition=0, offset=91 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:23:06', 'start': '2019-10-16 19:17:22',

json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:26:21', 'start': '2019-10-16 19:17:22', 'time_total': '0H 8M 59S', 'diff': 539, 'pace': 120.0, 'distance': 4.409999999999998}
created
topic=pace3, partition=0, offset=145 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:26:24', 'start': '2019-10-16 19:17:22', 'pace': 476.19, 'time_total': '0H 9M 2S', 'diff': 542, 'distance': 4.429999999999998})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:26:24', 'start': '2019-10-16 19:17:22', 'time_total': '0H 9M 2S', 'diff': 542, 'pace': 476.19, 'distance': 4.429999999999998}
created
topic=pace3, partition=0, offset=146 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:26:28', 'start': '2019-10-16 19:17:22', 'pace': 335.2, 'time_total': '0H 9M 6S', 'diff': 546, 'distance': 4.459999999999998})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:26:28', 'start': '2019-10-16 19:17:22', 'time_total': '0H 9M 6S', 'diff': 546, 'pace': 335.2, 'dist

json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:29:34', 'start': '2019-10-16 19:17:22', 'time_total': '0H 12M 12S', 'diff': 732, 'pace': 133.63, 'distance': 9.139999999999995}
created
topic=pace3, partition=0, offset=196 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:29:37', 'start': '2019-10-16 19:17:22', 'pace': 120.72, 'time_total': '0H 12M 15S', 'diff': 735, 'distance': 9.239999999999995})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:29:37', 'start': '2019-10-16 19:17:22', 'time_total': '0H 12M 15S', 'diff': 735, 'pace': 120.72, 'distance': 9.239999999999995}
created
topic=pace3, partition=0, offset=197 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-16 19:29:41', 'start': '2019-10-16 19:17:22', 'pace': 124.22, 'time_total': '0H 12M 19S', 'diff': 739, 'distance': 9.339999999999995})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-16 19:29:41', 'start': '2019-10-16 19:17:22', 'time_total': '0H 12M 19S', 'diff': 739, 'pace': 1

...

created
topic=pace3, partition=0, offset=2308 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:33:24', 'start': '2019-10-17 11:50:56', 'pace': 99.5, 'time_total': '00:42:27', 'diff': 2547, 'distance': 106.03})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:33:24', 'start': '2019-10-17 11:50:56', 'time_total': '00:42:27', 'diff': 2547, 'pace': 99.5, 'distance': 106.03}
created
topic=pace3, partition=0, offset=2309 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:33:27', 'start': '2019-10-17 11:50:56', 'pace': 38.27, 'time_total': '00:42:31', 'diff': 2551, 'distance': 107.14})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:33:27', 'start': '2019-10-17 11:50:56', 'time_total': '00:42:31', 'diff': 2551, 'pace': 38.27, 'distance': 107.14}
created
topic=pace3, partition=0, offset=2310 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:33:33', 'start': '2019-10-17 11:50:56', 'pace': 117.19, 'time_total': '00:42:36', '

json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:37:35', 'start': '2019-10-17 11:50:56', 'time_total': '00:46:38', 'diff': 2798, 'pace': 34.17, 'distance': 129.98}
created
topic=pace3, partition=0, offset=2371 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:37:38', 'start': '2019-10-17 11:50:56', 'pace': 39.58, 'time_total': '00:46:41', 'diff': 2801, 'distance': 131.16})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:37:38', 'start': '2019-10-17 11:50:56', 'time_total': '00:46:41', 'diff': 2801, 'pace': 39.58, 'distance': 131.16}
created
topic=pace3, partition=0, offset=2372 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:37:42', 'start': '2019-10-17 11:50:56', 'pace': 64.79, 'time_total': '00:46:45', 'diff': 2805, 'distance': 131.88})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:37:42', 'start': '2019-10-17 11:50:56', 'time_total': '00:46:45', 'diff': 2805, 'pace': 64.79, 'distance': 131.88}
created
topic=pace3, part

created
topic=pace3, partition=0, offset=2430 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:41:31', 'start': '2019-10-17 11:50:56', 'pace': 67.26, 'time_total': '00:50:35', 'diff': 3035, 'distance': 168.92000000000007})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:41:31', 'start': '2019-10-17 11:50:56', 'time_total': '00:50:35', 'diff': 3035, 'pace': 67.26, 'distance': 168.92000000000007}
created
topic=pace3, partition=0, offset=2431 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:41:36', 'start': '2019-10-17 11:50:56', 'pace': 103.63, 'time_total': '00:50:39', 'diff': 3039, 'distance': 169.41000000000008})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:41:36', 'start': '2019-10-17 11:50:56', 'time_total': '00:50:39', 'diff': 3039, 'pace': 103.63, 'distance': 169.41000000000008}
created
topic=pace3, partition=0, offset=2432 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:41:40', 'start': '2019-10-17 11

topic=pace3, partition=0, offset=2473 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:44:46', 'start': '2019-10-17 12:44:20', 'pace': 203.4, 'time_total': '00:00:26', 'diff': 26, 'distance': 0.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:44:46', 'start': '2019-10-17 12:44:20', 'time_total': '00:00:26', 'diff': 26, 'pace': 203.4, 'distance': 0.0}
created
topic=pace3, partition=0, offset=2474 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:44:50', 'start': '2019-10-17 12:44:20', 'pace': 1176.5, 'time_total': '00:00:29', 'diff': 29, 'distance': 0.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:44:50', 'start': '2019-10-17 12:44:20', 'time_total': '00:00:29', 'diff': 29, 'pace': 1176.5, 'distance': 0.0}
created
topic=pace3, partition=0, offset=2475 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:44:53', 'start': '2019-10-17 12:44:20', 'pace': 210.5, 'time_total': '00:00:33', 'diff': 33, 'distance': 0.

topic=pace3, partition=0, offset=2494 : (key=None, value={'device_id': 'yoyo', 'timestamp': '2019-10-17 12:46:11', 'start': '2019-10-17 12:44:20', 'pace': 1052.6, 'time_total': '00:01:50', 'diff': 110, 'distance': 0.0})
json= {'device_id': 'yoyo', 'timestamp': '2019-10-17 12:46:11', 'start': '2019-10-17 12:44:20', 'time_total': '00:01:50', 'diff': 110, 'pace': 1052.6, 'distance': 0.0}
created


In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch('http://elasticsearch:9200')
result = es.ping()
print(result)